<a href="https://colab.research.google.com/github/F1ameX/Modern-Methods-of-Deep-Machine-Learning/blob/main/1_polynominal_regression/1_polynominal_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt

In [26]:
N = 100 # size of sample
sample = np.empty((N, 2), dtype = float)
epsilons = [0.001, 0.01, 0.1, 1] # epsilon variability
rng = np.random.default_rng() # random generator
a, b, c, d = rng.uniform(low = -3.0, high = 3.0, size = 4) # random coefficient

In [27]:
def f(x : float, option : int = 1):
  global a, b, c, d

  if option == 1:
    return a * np.pow(x, 3) + b * np.pow(x, 2) + c * x + d

  elif option == 2:
    return x * np.sin(2 * np.pi * x)

In [25]:
for i in range(N):
  sample[i][0] = rng.uniform(low = -1.0, high = 1.0)

for func in range(2)
  for distribution in range(2):
    for eps in epsilons:
      for i in range(N):

        if distribution == 0:
          sample[i][1] = f(sample[i][0], option = func) + rng.uniform(low = -eps, high = eps)

        else:
          sample[i][1] = f(sample[i][0], option = func) + np.clip(rng.normal(loc = 0.0, scale = eps / 3), -eps, eps)